In [ ]:
# Importing deep learning 
from ultralytics import YOLO

# Other functionlaities 
import os 
import pandas as pd 
import numpy as np 
import math 

# Computer vision libs
import cv2 
from PIL import Image
from PIL.ExifTags import TAGS
import exifread

# Ploting 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# Loading the trained model 
model = YOLO('../runs/segment/train15/weights/last.pt')

In [ ]:
# Defining the path to the image 
path_to_img = 'input/test_image.JPG'
base_name = os.path.basename(path_to_img)

# Reading and ploting the image 
img = cv2.imread(path_to_img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

## Predicting pixels 

Once the model is loaded, we can extract the segments using the model object we just loaded. 

In [ ]:
# Applying the model 
results = model(path_to_img)

In [ ]:
# Extracting the image shape 
img_w, img_h = img.shape[1], img.shape[0]

# Extracting the segments 
segments = results[0].masks.xyn
print(f"Number of segments: {len(segments)}")

The above `segments` object is a list containing each found patch of the sosnovski plant. The coordinates are normalized to the original image size. 

An example patch is shown below. 

```python
[
    array([[   0.009375,      0.2375],
        [   0.009375,     0.26875],
        [   0.023438,     0.26875],
        [   0.023438,      0.2625],
        [      0.025,     0.26042],
        [      0.025,     0.25833],
        [   0.026563,     0.25625],
        [   0.032812,     0.25625],
        [   0.032812,      0.2375]], dtype=float32)
]
```

The first coordinate is the x-axis and the second coordinate is the y-axis. To plot on the image, we need to multiply the coordinates by the original image size. 

In [ ]:
# Converting to coordinates in pixels on the given image
segments_pixels = []
for segment in segments:
    segment_pixels = [(x * img_w, y * img_h) for x, y in segment]
    segments_pixels.append(segment_pixels)

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(img)

# Create and add a Polygon patch for each segment with red fill and border
for segment in segments_pixels:
    # Setting facecolor to red fills the polygon, and alpha adds transparency
    polygon = patches.Polygon(segment, closed=True, edgecolor='red', facecolor='red', linewidth=4, alpha=0.35)
    ax.add_patch(polygon)

plt.axis('off')
plt.show()